# Importing packages

In [ ]:
%config InlineBackend.figure_format = 'retina'
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from typing import Optional
%matplotlib inline


np.random.seed(42)

DIRECTORY = "./data/"

def encodeBind(df: pd.DataFrame, features: list) -> pd.DataFrame:
    """
    One-hot encoding the categorical variables in dataframe

    Parameters
    ----------
    df: pd.DataFrame
    The dataframe that need to be encoded.

    features: list
    A list of categorical variables that need to be encoded.

    Returns
    -------
    A one-hot encoded dataframe.
    """
    dummies = pd.get_dummies(df[features])
    resDF = pd.concat([df, dummies], axis=1)
    resDF.drop(features, axis=1, inplace = True)
    return resDF

def preprocessDF(df: pd.DataFrame, 
            featureScale:list, 
            featureEncode: Optional[list] = None) -> pd.DataFrame:
    """
    One-hot encoding the categorical variables in dataframe.

    Parameters
    ----------
    df: pd.DataFrame
    The dataframe that need to be preprocessed.

    featureScale: list
    A list of quantitative variables that need to be scaled.

    featureEncode: list, optional
    A list of categorical variables that need to be encoded.

    Returns
    -------
    A preprocessed dataframe with quantitative variables
    scaled and categorical variables one-hot encoded.
    """
    scaler = StandardScaler()
    dfLabel = df.goal

    df.drop("goal", axis = 1, inplace = True)
    if featureEncode != None:
        df = encodeBind(df, featureEncode)
    df["goal"] = dfLabel

    df[featureScale] = scaler.fit_transform(df[featureScale])
    
    return df

# Data clean-up

## Adult dataset

In [ ]:
adultNames = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status", 
              "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", 
              "hours-per-week", "native-country", "goal"]
adultEncode = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "sex", "native-country"]
adultScale = ["age", "fnlwgt", "education-num", "capital-gain", "capital-loss", "hours-per-week"]

adult = pd.read_table(
        os.path.join(DIRECTORY +"adult.data"),
        names = adultNames,
        sep = ",\s", 
        engine='python')
adult = adult[adult["workclass"] != "?"]
adult["goal"] = np.where(adult.goal == ">50K", 1, 0)

In [ ]:
adultScatterMat = pd.DataFrame(adult[adultScale], 
        columns = adultScale)
adultScatterMat["target"] = adult["goal"].values

sns.pairplot(adultScatterMat, hue = "target", diag_kind="hist")

In [ ]:
adult = preprocessDF(adult, adultScale, adultEncode)
adult.head()

## Covertype dataset

In [ ]:
covNames = ["elevation", "aspect", "slope", "hordishydro", "verdishydro", "hordisroad", "hillam", "hillnoon", "hillpm", 
            "hordisfire"] + ["wild" + str(i) for i in range(1,5)] + ["soil" + str(i) for i in range(1,41)] + ["goal"]
covScale = ["elevation", "aspect", "slope", "hordishydro", "verdishydro", "hordisroad", "hillam", "hillnoon", "hillpm", 
            "hordisfire"]

cov = pd.read_table(
        os.path.join(DIRECTORY +"covtype.data"), 
        sep = ",", 
        names = covNames)
cov.goal = np.where(cov.goal == cov["goal"].value_counts().idxmax(), 1, 0)

In [ ]:
covScatterMat = pd.DataFrame(cov[covScale], 
        columns = covScale)
covScatterMat["target"] = cov["goal"].values

sns.pairplot(covScatterMat, hue = "target", diag_kind="hist")

In [ ]:
cov = preprocessDF(cov, covScale, None)
cov.head()

## Letter dataset

In [ ]:
letterNames = ["goal", "x-box", "y-box", "width", "height", "onpix", "x-bar",
              "y-bar", "x2bar", "y2bar", "xybar", "x2ybr", "xy2br", "x-ege",
              "xegvy", "y-ege", "yegvx"]
letterScale = ["x-box", "y-box", "width", "height", "onpix", "x-bar",
              "y-bar", "x2bar", "y2bar", "xybar", "x2ybr", "xy2br", "x-ege",
              "xegvy", "y-ege", "yegvx"]

letter = pd.read_table(
        os.path.join(DIRECTORY + "letter-recognition.data"), 
        sep = ",", 
        names = letterNames)
letterCols = list(letter.columns)
letterCols[-1], letterCols[0] = letterCols[0], letterCols[-1]
letter = letter[letterCols]
chosenLetter = [chr(i) for i in range(ord('A'), ord('M')+1)]
letter.goal = letter.goal.apply(lambda x: 1 if x in chosenLetter else 0)

In [ ]:
letterScatterMat = pd.DataFrame(letter[letterScale], 
        columns = letterScale)
letterScatterMat["target"] = letter["goal"].values

sns.pairplot(letterScatterMat, hue = "target", diag_kind="hist")

In [ ]:
letter = preprocessDF(letter, letterScale, None)
letter.head()

## Avila dataset

In [ ]:
avilaNames = ["interdis", "upmar", "lowmar", "exploi", "rownum", "modratio",
             "interspace", "weight", "peaknum", "modratio-interspace", "goal"]

avila = pd.read_csv(os.path.join(DIRECTORY +"avila.txt"), 
        sep = ",", 
        names = avilaNames)
avilaScale = list(avila.columns)[:-1]
avila.goal = np.where(avila.goal == "A", 1, 0)

In [ ]:
avilaScatterMat = pd.DataFrame(avila[avilaScale], 
        columns = avilaScale)
avilaScatterMat["target"] = avila["goal"].values

sns.pairplot(avilaScatterMat, hue = "target", diag_kind="hist")

In [ ]:
avila.head()

## Bean dataset

In [ ]:
bean = pd.read_excel(os.path.join(DIRECTORY +"Dry_Bean_Dataset.xlsx"), 
        engine='openpyxl')
        
beanScale = list(bean.columns)[:-1]
bean.Class = np.where(bean.Class == bean.Class.value_counts().idxmax(), 1, 0)
bean.columns = [*bean.columns[:-1], 'goal']

In [ ]:
beanScatterMat = pd.DataFrame(bean[beanScale], 
        columns = beanScale)
beanScatterMat["target"] = bean["goal"].values

sns.pairplot(beanScatterMat, hue = "target", diag_kind="hist")

In [ ]:
bean = preprocessDF(bean, beanScale)
bean.head()

# Function definition for main program

In [ ]:
def get_trail_train_test(pipe: Pipeline, 
                param: dict, 
                score_function, 
                X_train, 
                X_test, 
                Y_train, 
                Y_test):
    """
    Given a pipeline and the parameters, the function will give the 
    metric score of the training and testing prediction.

    Parameters
    ----------
    pipe: Pipeline
    An algorithm model pipeline.

    param: dict
    The parameters the algorithm used for grid searching.

    score_function: sklearn.metrics
    A metric function to give a score.

    X_train, X_test, Y_train, Y_test: np.ndarray
    Training and testing data with labels already splitted.

    Returns
    -------
    Return the score of the metrics given for the training and testing set.
    """                
    pipe.set_params(**param)
    pipe.fit(X_train, Y_train)
    y_pred_train = pipe.predict(X_train)
    y_pred_test = pipe.predict(X_test)
    trialTrain = score_function(Y_train, y_pred_train)
    trialTest = score_function(Y_test, y_pred_test)
    
    return trialTrain, trialTest

def spit_out_res(data: pd.DataFrame, 
                pipe: Pipeline, 
                param: dict):
    """
    The function will compute the metrics (in this case, AUC, 
    accuracy and F1-score) from grid search cross-validation.

    Parameters
    ----------
    data: pd.DataFrame
    Training and testing data with labels in a dataframe.

    pipe: Pipeline
    An algorithm model pipeline.

    param: dict
    The parameters the algorithm used for grid searching.

    Returns
    -------
    Return the AUC, accuracy and F1-score of the given data 
    with a specific algorithm and parameter. 
    """
    aucTrialTrain = []
    accTrialTrain = []
    f1TrialTrain = []
    aucTrialTest = []
    accTrialTest = []
    f1TrialTest = []
    for trial in range(5):
        print(trial)
        X, Y = data.iloc[:,:-1], data.iloc[:,-1]
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=5000, shuffle=True)
        gs = GridSearchCV(pipe, param_grid = param, cv = StratifiedKFold(), n_jobs = -1, 
                          scoring = scoring, refit = False)
        gs.fit(X_train, Y_train)
        results = gs.cv_results_
    
        auc_best_index = np.argmin(results['rank_test_AUC'])
        auc_best_param = results['params'][auc_best_index]
    
        accuracy_best_index = np.argmin(results['rank_test_Accuracy'])
        accuracy_best_param = results['params'][accuracy_best_index]
    
        f1_best_index = np.argmin(results['rank_test_F1'])
        f1_best_param = results['params'][f1_best_index]
    
        aucTr, aucTest = get_trail_train_test(pipe, auc_best_param, roc_auc_score, X_train, X_test, Y_train, Y_test)
        accTr, accTest = get_trail_train_test(pipe, accuracy_best_param, accuracy_score, X_train, X_test, Y_train, Y_test)
        f1Tr, f1Test = get_trail_train_test(pipe, f1_best_param, f1_score, X_train, X_test, Y_train, Y_test)
        
        aucTrialTrain.append(aucTr)
        accTrialTrain.append(accTr)
        f1TrialTrain.append(f1Tr)
        aucTrialTest.append(aucTest)
        accTrialTest.append(accTest)
        f1TrialTest.append(f1Test)
        
    return aucTrialTrain, accTrialTrain, f1TrialTrain, aucTrialTest, accTrialTest, f1TrialTest

def run_per_algo(algoPipe: Pipeline, 
                datasets: list, 
                param: dict):
    """
    The function will calculate the raw and mean test set performance across 
    trails for an algorithm with all datasets over three metrics. Furthermore,
    it also includes the training set performance.

    Parameters
    ----------
    algoPipe: Pipeline
    An algorithm model pipeline.

    datasets: list
    A list of datasets which we will iterate over.

    param: dict
    The parameters the algorithm used for grid searching.

    Returns
    -------
    Return 6 tables for raw and mean training and testing score over 3 metrics.
    """
    aucTrainMean, accTrainMean, f1TrainMean, aucTestMean, accTestMean, f1TestMean = [], [], [], [], [], []
    aucTrainRaw, accTrainRaw, f1TrainRaw, aucTestRaw, accTestRaw, f1TestRaw = [], [], [], [], [], []
    for data in datasets:
        aucTrain, accTrain, f1Train, aucTest, accTest, f1Test = spit_out_res(data, algoPipe, param)

        accTrainRaw.append(accTrain)
        accTestRaw.append(accTest)
        
        aucTrainMean.append(np.mean(aucTrain))
        accTrainMean.append(np.mean(accTrain))
        f1TrainMean.append(np.mean(f1Train))
        aucTestMean.append(np.mean(aucTest))
        accTestMean.append(np.mean(accTest))
        f1TestMean.append(np.mean(f1Test))
        
    print("Raw train score")

    accTrainDF = pd.DataFrame(accTrainRaw, columns = ["Trial1", "Trial2", "Trial3", "Trial 4", "Trial 5"], dtype = float)
    print(accTrainDF)
    
    print("Raw test score")

    accTestDF = pd.DataFrame(accTestRaw, columns = ["Trial1", "Trial2", "Trial3", "Trial 4", "Trial 5"], dtype = float)
    print(accTestDF)
    
    theDFTest = pd.DataFrame(list(zip(aucTestMean, accTestMean, f1TestMean)), columns = ["AUC", "ACC", "F1"], dtype = float)
    theDFTrain = pd.DataFrame(list(zip(aucTrainMean, accTrainMean, f1TrainMean)), columns = ["AUC", "ACC", "F1"], dtype = float)
    
    print("Test over problems")
    print(theDFTest.mean(axis = 1))
    print("Test over metrics")
    print(theDFTest.mean(axis = 0))
    
    print("Train over problems")
    print(theDFTrain.mean(axis = 1))
    print("Train over metrics")
    print(theDFTrain.mean(axis = 0))

# Parameters for grid search cross-validation

In [ ]:
scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score), 'F1': make_scorer(f1_score)}

knnParam = {'knn__n_neighbors': np.arange(1,106,4),
    'knn__weights': ["uniform", "distance"]}

nnParam = [{
    "nn__hidden_layer_sizes": [(1,), (2,), (4,), (8,), (32,), (128,)],
    "nn__solver": ["adam"]
}, {
    "nn__hidden_layer_sizes": [(1,), (2,), (4,), (8,), (32,), (128,)],
    "nn__solver": ["sgd"],
    "nn__momentum": [0, 0.2, 0.5, 0.9]
}]

rfParam = {
    'rf__max_features': ["auto", "log2", None, 1, 2, 4, 6, 8, 10]
    }

lrParam = [{
    'lr__solver': ['saga'],
    'lr__penalty': ['l1', 'l2'],
    'lr__C': [10**i for i in range(-8,5)]
    }, {
    'lr__solver': ['lbfgs'],
    'lr__penalty': ['l2'],
    'lr__C': [10**i for i in range(-8,5)]
    }, {
    'lr__solver': ['lbfgs','saga'],
    'lr__penalty': ['none'],
}]

knnPipe = Pipeline([("knn", KNeighborsClassifier())])
nnPipe = Pipeline([("nn", MLPClassifier(max_iter=500))])
rfPipe = Pipeline([("rf", RandomForestClassifier(n_estimators = 1024, n_jobs = -1))])
lrPipe = Pipeline([("lr", LogisticRegression(max_iter = 5000))])

params = [knnParam, nnParam, rfParam, lrParam]
pipes = [knnPipe, nnPipe, rfPipe, lrPipe]
datasets = [adult, cov, letter, avila, bean]

# Program run

## K nearest neighbors model

In [ ]:
run_per_algo(knnPipe, datasets, knnParam)

## MLP model

In [ ]:
run_per_algo(nnPipe, datasets, nnParam)

## Random Forest Model

In [ ]:
run_per_algo(rfPipe, datasets, rfParam)

## Logistic Regression Model

In [ ]:
run_per_algo(lrPipe, datasets, lrParam)